In [0]:
# provider-notebooks/Module 2 - Delta Sharing Deep Dive/Includes/Demo-Setup-2.ipynb

In [0]:
%run ./_common

In [0]:
# set variables in python and sql
config_values = [
    ("catalog", "acme_corp"),
    ("schema", "retail"),
    ("internal_share_name", "internal_retail"),
    ("internal_recipient_name", "west_division"),
    ("external_share_name", "partner_retail"),
    ("external_recipient_name", "partner_co"),
]
DA = create_da_objects(config_values)

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS IDENTIFIER(DA.catalog)
COMMENT 'Provider catalog for Delta Sharing demo';
USE CATALOG IDENTIFIER(DA.catalog);

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS IDENTIFIER(DA.schema)
COMMENT 'Schema containing sales and customer data for sharing';
USE SCHEMA IDENTIFIER(DA.schema);

In [0]:
# Cleanup existing shares and recipients before setup
cleanup_commands = [
    "DROP RECIPIENT IF EXISTS west_division",
    "DROP RECIPIENT IF EXISTS partner_co",
    "DROP SHARE IF EXISTS internal_retail",
    "DROP SHARE IF EXISTS partner_retail", 
]

for cmd in cleanup_commands:
    try:
        spark.sql(cmd)
    except Exception as e:
        print(f"Note: {cmd} - {str(e)}")

In [0]:
%sql
-- Drop table if exists (to keep history clean)
DROP TABLE IF EXISTS customers;

-- Create customers table
CREATE TABLE customers (
  customer_id INT,
  customer_name STRING,
  email STRING,
  country STRING,
  signup_date DATE,
  customer_segment STRING
)
COMMENT 'Customer master data';

-- Insert sample data - More realistic distribution for better visualizations
INSERT INTO customers VALUES
  -- Enterprise customers (20%)
  (1, 'Acme Corp', 'contact@acme.com', 'USA', '2023-01-15', 'Enterprise'),
  (3, 'Global Solutions', 'hello@globalsol.com', 'UK', '2023-03-10', 'Enterprise'),
  (5, 'Data Dynamics', 'team@datadyn.com', 'Germany', '2023-05-12', 'Enterprise'),
  (8, 'Digital Transform', 'hello@digitrans.com', 'France', '2023-08-30', 'Enterprise'),
  (12, 'Mega Industries', 'contact@megaind.com', 'USA', '2023-10-05', 'Enterprise'),
  (18, 'Continental Bank', 'info@contbank.com', 'UK', '2024-01-10', 'Enterprise'),
  (25, 'Pacific Group', 'hello@pacific.com', 'Australia', '2024-03-20', 'Enterprise'),
  (32, 'Euro Solutions', 'contact@eurosol.com', 'Germany', '2024-06-15', 'Enterprise'),
  -- Mid-Market customers (30%)
  (4, 'Innovate Ltd', 'contact@innovate.com', 'Australia', '2023-04-05', 'Mid-Market'),
  (7, 'NextGen Analytics', 'contact@nextgen.com', 'Singapore', '2023-07-22', 'Mid-Market'),
  (10, 'Smart Systems', 'team@smartsys.com', 'Canada', '2023-09-15', 'Mid-Market'),
  (14, 'Fast Track Inc', 'info@fasttrack.com', 'USA', '2023-11-20', 'Mid-Market'),
  (16, 'Blue Ocean Tech', 'contact@blueocean.com', 'UK', '2023-12-05', 'Mid-Market'),
  (20, 'Apex Consulting', 'hello@apex.com', 'Singapore', '2024-02-08', 'Mid-Market'),
  (23, 'Urban Analytics', 'team@urban.com', 'Canada', '2024-03-01', 'Mid-Market'),
  (27, 'Peak Performance', 'info@peakperf.com', 'Australia', '2024-04-22', 'Mid-Market'),
  (30, 'Horizon Group', 'contact@horizon.com', 'USA', '2024-05-30', 'Mid-Market'),
  (34, 'Quantum Insights', 'hello@quantum.com', 'UK', '2024-07-10', 'Mid-Market'),
  (37, 'Velocity Corp', 'team@velocity.com', 'Singapore', '2024-08-05', 'Mid-Market'),
  (40, 'Summit Solutions', 'info@summit.com', 'Canada', '2024-09-12', 'Mid-Market'),
  -- SMB customers (50%)
  (2, 'TechStart Inc', 'info@techstart.com', 'Canada', '2023-02-20', 'SMB'),
  (6, 'Cloud First', 'info@cloudfirst.com', 'USA', '2023-06-18', 'SMB'),
  (9, 'Pixel Perfect', 'hello@pixelperfect.com', 'UK', '2023-09-01', 'SMB'),
  (11, 'Code Craft', 'team@codecraft.com', 'USA', '2023-09-28', 'SMB'),
  (13, 'Data Labs', 'contact@datalabs.com', 'Germany', '2023-11-08', 'SMB'),
  (15, 'Agile Solutions', 'info@agilesol.com', 'France', '2023-11-30', 'SMB'),
  (17, 'Bright Ideas', 'hello@brightideas.com', 'Australia', '2024-01-05', 'SMB'),
  (19, 'Swift Systems', 'team@swift.com', 'USA', '2024-01-25', 'SMB'),
  (21, 'Creative Minds', 'contact@creative.com', 'Canada', '2024-02-14', 'SMB'),
  (22, 'Tech Ventures', 'info@techvent.com', 'UK', '2024-02-28', 'SMB'),
  (24, 'Digital Edge', 'hello@digedge.com', 'Singapore', '2024-03-18', 'SMB'),
  (26, 'Smart Cloud', 'team@smartcloud.com', 'USA', '2024-04-10', 'SMB'),
  (28, 'Rapid Deploy', 'contact@rapiddeploy.com', 'France', '2024-05-05', 'SMB'),
  (29, 'Nimble Tech', 'info@nimbletech.com', 'Germany', '2024-05-20', 'SMB'),
  (31, 'Spark Innovations', 'hello@spark.com', 'Australia', '2024-06-08', 'SMB'),
  (33, 'Fusion Data', 'team@fusiondata.com', 'USA', '2024-06-28', 'SMB'),
  (35, 'Pulse Analytics', 'contact@pulse.com', 'UK', '2024-07-15', 'SMB'),
  (36, 'Shift Digital', 'info@shift.com', 'Canada', '2024-07-30', 'SMB'),
  (38, 'Nexus Systems', 'hello@nexus.com', 'Singapore', '2024-08-18', 'SMB'),
  (39, 'Catalyst Tech', 'team@catalyst.com', 'France', '2024-09-01', 'SMB');

In [0]:
%sql
-- Drop table if exists (to keep history clean)
DROP TABLE IF EXISTS sales_transactions;

-- Create sales transactions table
CREATE TABLE sales_transactions (
  transaction_id INT,
  customer_id INT,
  product_name STRING,
  quantity INT,
  unit_price DECIMAL(10,2),
  total_amount DECIMAL(10,2),
  transaction_date DATE,
  region STRING
)
COMMENT 'Sales transaction data';

-- Insert realistic sales data with varied distribution
-- Enterprise customers: high volume, multiple transactions
INSERT INTO sales_transactions VALUES
  -- Acme Corp (customer 1) - 15 transactions
  (101, 1, 'Data Platform License', 5, 1000.00, 5000.00, '2024-01-15', 'North America'),
  (104, 1, 'Storage Add-on', 3, 200.00, 600.00, '2024-02-10', 'North America'),
  (112, 1, 'Analytics Suite', 8, 500.00, 4000.00, '2024-03-05', 'North America'),
  (120, 1, 'Data Platform License', 10, 1000.00, 10000.00, '2024-04-12', 'North America'),
  (128, 1, 'ML Add-on', 5, 800.00, 4000.00, '2024-05-20', 'North America'),
  (136, 1, 'Storage Add-on', 7, 200.00, 1400.00, '2024-06-15', 'North America'),
  (145, 1, 'Analytics Suite', 12, 500.00, 6000.00, '2024-07-08', 'North America'),
  (153, 1, 'Data Platform License', 15, 1000.00, 15000.00, '2024-08-14', 'North America'),
  (162, 1, 'Streaming Add-on', 6, 1200.00, 7200.00, '2024-09-05', 'North America'),
  (171, 1, 'Storage Add-on', 10, 200.00, 2000.00, '2024-10-01', 'North America'),
  (180, 1, 'ML Add-on', 8, 800.00, 6400.00, '2024-10-18', 'North America'),
  (189, 1, 'Analytics Suite', 15, 500.00, 7500.00, '2024-11-02', 'North America'),
  (198, 1, 'Data Platform License', 20, 1000.00, 20000.00, '2024-11-20', 'North America'),
  (207, 1, 'Storage Add-on', 12, 200.00, 2400.00, '2024-12-05', 'North America'),
  (216, 1, 'Streaming Add-on', 10, 1200.00, 12000.00, '2024-12-18', 'North America'),
  -- Global Solutions (customer 3) - 12 transactions
  (103, 3, 'Data Platform License', 10, 1000.00, 10000.00, '2024-02-05', 'EMEA'),
  (107, 3, 'Storage Add-on', 8, 200.00, 1600.00, '2024-03-10', 'EMEA'),
  (115, 3, 'Analytics Suite', 15, 500.00, 7500.00, '2024-04-01', 'EMEA'),
  (123, 3, 'ML Add-on', 10, 800.00, 8000.00, '2024-05-08', 'EMEA'),
  (132, 3, 'Data Platform License', 12, 1000.00, 12000.00, '2024-06-05', 'EMEA'),
  (141, 3, 'Streaming Add-on', 8, 1200.00, 9600.00, '2024-07-12', 'EMEA'),
  (150, 3, 'Storage Add-on', 15, 200.00, 3000.00, '2024-08-20', 'EMEA'),
  (159, 3, 'Analytics Suite', 20, 500.00, 10000.00, '2024-09-15', 'EMEA'),
  (168, 3, 'ML Add-on', 12, 800.00, 9600.00, '2024-10-10', 'EMEA'),
  (177, 3, 'Data Platform License', 18, 1000.00, 18000.00, '2024-11-05', 'EMEA'),
  (186, 3, 'Streaming Add-on', 10, 1200.00, 12000.00, '2024-11-25', 'EMEA'),
  (195, 3, 'Storage Add-on', 20, 200.00, 4000.00, '2024-12-10', 'EMEA'),
  -- Data Dynamics (customer 5) - 10 transactions
  (106, 5, 'Data Platform License', 15, 1000.00, 15000.00, '2024-03-01', 'EMEA'),
  (117, 5, 'Analytics Suite', 12, 500.00, 6000.00, '2024-04-15', 'EMEA'),
  (126, 5, 'Storage Add-on', 10, 200.00, 2000.00, '2024-05-20', 'EMEA'),
  (139, 5, 'ML Add-on', 8, 800.00, 6400.00, '2024-07-01', 'EMEA'),
  (148, 5, 'Data Platform License', 18, 1000.00, 18000.00, '2024-08-15', 'EMEA'),
  (157, 5, 'Streaming Add-on', 7, 1200.00, 8400.00, '2024-09-10', 'EMEA'),
  (166, 5, 'Analytics Suite', 15, 500.00, 7500.00, '2024-10-05', 'EMEA'),
  (175, 5, 'Storage Add-on', 12, 200.00, 2400.00, '2024-11-01', 'EMEA'),
  (184, 5, 'ML Add-on', 10, 800.00, 8000.00, '2024-11-22', 'EMEA'),
  (193, 5, 'Data Platform License', 20, 1000.00, 20000.00, '2024-12-12', 'EMEA'),
  -- Digital Transform (customer 8) - 8 transactions
  (110, 8, 'Storage Add-on', 10, 200.00, 2000.00, '2024-04-05', 'EMEA'),
  (119, 8, 'Data Platform License', 12, 1000.00, 12000.00, '2024-05-10', 'EMEA'),
  (130, 8, 'Analytics Suite', 10, 500.00, 5000.00, '2024-06-20', 'EMEA'),
  (143, 8, 'ML Add-on', 6, 800.00, 4800.00, '2024-08-01', 'EMEA'),
  (155, 8, 'Streaming Add-on', 5, 1200.00, 6000.00, '2024-09-15', 'EMEA'),
  (167, 8, 'Data Platform License', 15, 1000.00, 15000.00, '2024-10-20', 'EMEA'),
  (179, 8, 'Storage Add-on', 12, 200.00, 2400.00, '2024-11-15', 'EMEA'),
  (191, 8, 'Analytics Suite', 14, 500.00, 7000.00, '2024-12-08', 'EMEA'),
  -- Mega Industries (customer 12) - 9 transactions
  (125, 12, 'Data Platform License', 20, 1000.00, 20000.00, '2024-05-15', 'North America'),
  (134, 12, 'Storage Add-on', 15, 200.00, 3000.00, '2024-06-10', 'North America'),
  (147, 12, 'Analytics Suite', 18, 500.00, 9000.00, '2024-08-05', 'North America'),
  (156, 12, 'ML Add-on', 12, 800.00, 9600.00, '2024-09-12', 'North America'),
  (169, 12, 'Streaming Add-on', 10, 1200.00, 12000.00, '2024-10-15', 'North America'),
  (178, 12, 'Data Platform License', 22, 1000.00, 22000.00, '2024-11-10', 'North America'),
  (187, 12, 'Storage Add-on', 18, 200.00, 3600.00, '2024-11-28', 'North America'),
  (200, 12, 'Analytics Suite', 20, 500.00, 10000.00, '2024-12-15', 'North America'),
  (209, 12, 'ML Add-on', 15, 800.00, 12000.00, '2024-12-28', 'North America'),
  -- Mid-Market customers: moderate volume
  -- Innovate Ltd (customer 4) - 6 transactions
  (105, 4, 'Analytics Suite', 5, 500.00, 2500.00, '2024-02-15', 'APAC'),
  (121, 4, 'Data Platform License', 8, 1000.00, 8000.00, '2024-05-01', 'APAC'),
  (138, 4, 'Storage Add-on', 5, 200.00, 1000.00, '2024-07-15', 'APAC'),
  (158, 4, 'Analytics Suite', 7, 500.00, 3500.00, '2024-09-20', 'APAC'),
  (176, 4, 'ML Add-on', 4, 800.00, 3200.00, '2024-11-08', 'APAC'),
  (197, 4, 'Data Platform License', 10, 1000.00, 10000.00, '2024-12-20', 'APAC'),
  -- NextGen Analytics (customer 7) - 7 transactions
  (109, 7, 'Data Platform License', 7, 1000.00, 7000.00, '2024-04-01', 'APAC'),
  (127, 7, 'Analytics Suite', 6, 500.00, 3000.00, '2024-06-05', 'APAC'),
  (144, 7, 'Storage Add-on', 4, 200.00, 800.00, '2024-08-10', 'APAC'),
  (161, 7, 'ML Add-on', 5, 800.00, 4000.00, '2024-10-01', 'APAC'),
  (181, 7, 'Data Platform License', 9, 1000.00, 9000.00, '2024-11-18', 'APAC'),
  (199, 7, 'Streaming Add-on', 3, 1200.00, 3600.00, '2024-12-12', 'APAC'),
  (210, 7, 'Analytics Suite', 8, 500.00, 4000.00, '2024-12-28', 'APAC'),
  -- Smart Systems (customer 10) - 5 transactions
  (116, 10, 'Analytics Suite', 4, 500.00, 2000.00, '2024-04-20', 'North America'),
  (135, 10, 'Data Platform License', 6, 1000.00, 6000.00, '2024-07-05', 'North America'),
  (154, 10, 'Storage Add-on', 3, 200.00, 600.00, '2024-09-18', 'North America'),
  (174, 10, 'Analytics Suite', 5, 500.00, 2500.00, '2024-11-02', 'North America'),
  (194, 10, 'ML Add-on', 3, 800.00, 2400.00, '2024-12-15', 'North America'),
  -- Fast Track Inc (customer 14) - 5 transactions
  (129, 14, 'Data Platform License', 5, 1000.00, 5000.00, '2024-06-15', 'North America'),
  (146, 14, 'Analytics Suite', 4, 500.00, 2000.00, '2024-08-20', 'North America'),
  (163, 14, 'Storage Add-on', 2, 200.00, 400.00, '2024-10-10', 'North America'),
  (183, 14, 'Data Platform License', 7, 1000.00, 7000.00, '2024-11-25', 'North America'),
  (202, 14, 'Analytics Suite', 6, 500.00, 3000.00, '2024-12-18', 'North America'),
  -- Blue Ocean Tech (customer 16) - 4 transactions
  (137, 16, 'Analytics Suite', 3, 500.00, 1500.00, '2024-07-08', 'EMEA'),
  (160, 16, 'Data Platform License', 5, 1000.00, 5000.00, '2024-09-25', 'EMEA'),
  (185, 16, 'Storage Add-on', 2, 200.00, 400.00, '2024-11-20', 'EMEA'),
  (205, 16, 'Analytics Suite', 4, 500.00, 2000.00, '2024-12-22', 'EMEA'),
  -- SMB customers: lower volume, 1-3 transactions each
  -- TechStart Inc (customer 2) - 3 transactions
  (102, 2, 'Analytics Suite', 2, 500.00, 1000.00, '2024-01-20', 'North America'),
  (124, 2, 'Storage Add-on', 1, 200.00, 200.00, '2024-05-15', 'North America'),
  (170, 2, 'Analytics Suite', 3, 500.00, 1500.00, '2024-10-22', 'North America'),
  -- Cloud First (customer 6) - 2 transactions
  (108, 6, 'Analytics Suite', 1, 500.00, 500.00, '2024-03-15', 'North America'),
  (165, 6, 'Storage Add-on', 2, 200.00, 400.00, '2024-10-12', 'North America'),
  -- Pixel Perfect (customer 9) - 2 transactions
  (113, 9, 'Analytics Suite', 2, 500.00, 1000.00, '2024-04-10', 'EMEA'),
  (172, 9, 'Storage Add-on', 1, 200.00, 200.00, '2024-11-01', 'EMEA'),
  -- Code Craft (customer 11) - 3 transactions
  (118, 11, 'Data Platform License', 3, 1000.00, 3000.00, '2024-05-05', 'North America'),
  (151, 11, 'Analytics Suite', 2, 500.00, 1000.00, '2024-09-01', 'North America'),
  (196, 11, 'Storage Add-on', 1, 200.00, 200.00, '2024-12-10', 'North America'),
  -- Data Labs (customer 13) - 2 transactions
  (131, 13, 'Analytics Suite', 1, 500.00, 500.00, '2024-06-20', 'EMEA'),
  (182, 13, 'Data Platform License', 2, 1000.00, 2000.00, '2024-11-18', 'EMEA'),
  -- Agile Solutions (customer 15) - 1 transaction
  (140, 15, 'Analytics Suite', 2, 500.00, 1000.00, '2024-07-25', 'EMEA'),
  -- Bright Ideas (customer 17) - 2 transactions
  (142, 17, 'Storage Add-on', 1, 200.00, 200.00, '2024-08-05', 'APAC'),
  (188, 17, 'Analytics Suite', 2, 500.00, 1000.00, '2024-11-28', 'APAC'),
  -- Swift Systems (customer 19) - 3 transactions
  (149, 19, 'Data Platform License', 2, 1000.00, 2000.00, '2024-08-22', 'North America'),
  (173, 19, 'Analytics Suite', 1, 500.00, 500.00, '2024-11-05', 'North America'),
  (208, 19, 'Storage Add-on', 1, 200.00, 200.00, '2024-12-28', 'North America'),
  -- Creative Minds (customer 21) - 1 transaction
  (152, 21, 'Analytics Suite', 1, 500.00, 500.00, '2024-09-05', 'North America'),
  -- Tech Ventures (customer 22) - 2 transactions
  (164, 22, 'Data Platform License', 2, 1000.00, 2000.00, '2024-10-08', 'EMEA'),
  (203, 22, 'Analytics Suite', 1, 500.00, 500.00, '2024-12-20', 'EMEA'),
  -- Digital Edge (customer 24) - 1 transaction
  (190, 24, 'Analytics Suite', 2, 500.00, 1000.00, '2024-12-01', 'APAC'),
  -- Smart Cloud (customer 26) - 2 transactions
  (201, 26, 'Storage Add-on', 1, 200.00, 200.00, '2024-12-15', 'North America'),
  (211, 26, 'Analytics Suite', 2, 500.00, 1000.00, '2024-12-29', 'North America'),
  -- Rapid Deploy (customer 28) - 1 transaction
  (204, 28, 'Analytics Suite', 1, 500.00, 500.00, '2024-12-22', 'EMEA'),
  -- Nimble Tech (customer 29) - 1 transaction
  (206, 29, 'Data Platform License', 2, 1000.00, 2000.00, '2024-12-25', 'EMEA'),
  -- Spark Innovations (customer 31) - 1 transaction
  (212, 31, 'Analytics Suite', 1, 500.00, 500.00, '2024-12-30', 'APAC'),
  -- Fusion Data (customer 33) - 2 transactions
  (213, 33, 'Storage Add-on', 1, 200.00, 200.00, '2024-12-30', 'North America'),
  (214, 33, 'Analytics Suite', 1, 500.00, 500.00, '2024-12-31', 'North America'),
  -- Pulse Analytics (customer 35) - 1 transaction
  (215, 35, 'Data Platform License', 1, 1000.00, 1000.00, '2024-12-31', 'EMEA');

In [0]:
display_config_values(config_values)